In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests

class InvalidPointException(Exception):
    pass

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,2020-02-22 20:22:53,-54.80,-68.30,73.40,38,40,13.87
1,1,New Glasgow,CA,2020-02-22 20:25:23,45.58,-62.65,34.00,67,97,8.99
2,2,Katangli,RU,2020-02-22 20:25:24,51.71,143.23,10.94,97,97,3.60
3,3,Comodoro Rivadavia,AR,2020-02-22 20:25:24,-45.87,-67.50,80.60,26,26,21.92
4,4,Christchurch,NZ,2020-02-22 20:25:24,-43.53,172.63,60.01,93,0,3.36


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature - Google heatmaps do not plot (- numbers) 
# use for loop to change (-s to 0)
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable - list comprehension method weights=[max(temp, 0) for temp in max_temp])
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Comodoro Rivadavia,AR,2020-02-22 20:25:24,-45.87,-67.50,80.60,26,26,21.92
6,6,Rikitea,PF,2020-02-22 20:25:25,-23.12,-134.97,78.94,70,1,7.52
9,9,Atuona,PF,2020-02-22 20:25:26,-9.80,-139.03,81.86,76,0,13.73
12,12,Hilo,US,2020-02-22 20:25:26,19.73,-155.09,77.00,65,20,6.93
18,18,Belmonte,BR,2020-02-22 20:25:28,-15.86,-38.88,86.00,79,20,10.29
19,19,Mahebourg,MU,2020-02-22 20:25:28,-20.41,57.70,77.00,88,40,2.24
22,22,Sola,VU,2020-02-22 20:25:29,-13.88,167.55,81.86,80,24,14.54
29,29,Souillac,MU,2020-02-22 20:25:30,-20.52,57.52,77.00,88,40,2.24
35,35,San Andres,CO,2020-02-22 20:21:00,12.58,-81.70,86.00,62,20,8.05
36,36,Georgetown,MY,2020-02-22 20:22:41,5.41,100.34,80.60,78,20,12.75


In [18]:
preferred_cities_df.count()

City_ID       154
City          154
Country       154
Date          154
Lat           154
Lng           154
Max Temp      154
Humidity      154
Cloudiness    154
Wind Speed    154
dtype: int64

In [20]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Comodoro Rivadavia,AR,80.60,-45.87,-67.50,
6,Rikitea,PF,78.94,-23.12,-134.97,
9,Atuona,PF,81.86,-9.80,-139.03,
12,Hilo,US,77.00,19.73,-155.09,
18,Belmonte,BR,86.00,-15.86,-38.88,
19,Mahebourg,MU,77.00,-20.41,57.70,
22,Sola,VU,81.86,-13.88,167.55,
29,Souillac,MU,77.00,-20.52,57.52,
35,San Andres,CO,86.00,12.58,-81.70,
36,Georgetown,MY,80.60,5.41,100.34,


In [25]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'CqQCHgEAAM5qs86eLG-UDYayhl0e32qwaZ3rv8yNJS8MMIvN4fJjKKcOnsVmiaTG9HQp9Gblbpluz3Hld-PFI-6_0yEN_sE12ngqxYiz53KMuFZo3HACG-WahtC6QHurcVhOkqKIQ8PGJ1566yd3k63Pp5E5B5qIy6HTSGu8_OfcUbxWJQc3Su6d9iY0rZr0J29k4j54L2OvofcPCwEaQxSFJ8yj9FQm9mW763mpxQEmWvHyn_HDKThu3JkscPDlsWwnM8vyyJjrSPRq2Y4Dz3BIhmsnwoCpMe0l2MQCanJRLMYJ60uvA_Rf2eFNelzsHSq-e-ZxxkFyFP4aQRd8VEY9Lvh5NwbpvJDCEtHoqBrBK4jzo1dfXIb1PhYd314NBhoAaosHnxIQ32dD8yY-mv1UcsgCSof-aBoUQ5m6c8WBtAw_crJmytXIxUfYbFQ',
 'results': [{'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594402302915,
      'lng': 2.354348530291502},
     'southwest': {'lat': 48.8567422697085, 'lng': 2.351650569708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '7fa47cb61d7345cecc3b802d0e04fcff2b3d5ecb',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,
     'html_attributions': ['<a href=

In [26]:
len(hotels["results"])

20

In [29]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Comodoro Rivadavia,AR,80.60,-45.87,-67.50,Austral Hotel
6,Rikitea,PF,78.94,-23.12,-134.97,Pension Maro'i
9,Atuona,PF,81.86,-9.80,-139.03,Villa Enata
12,Hilo,US,77.00,19.73,-155.09,Hilo Hawaiian Hotel
18,Belmonte,BR,86.00,-15.86,-38.88,Pousada Monte Carmelo O Pão
19,Mahebourg,MU,77.00,-20.41,57.70,Shandrani Beachcomber Resort & Spa
22,Sola,VU,81.86,-13.88,167.55,Leumerus Bungalows
29,Souillac,MU,77.00,-20.52,57.52,Bils' Villa
35,San Andres,CO,86.00,12.58,-81.70,Hotel Arena Blanca
36,Georgetown,MY,80.60,5.41,100.34,Cititel Penang


In [35]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [37]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))